In [3]:
import pandas as pd
import numpy as np
import joblib as jb
from scipy.stats import spearmanr

In [4]:
train_train_level0 = jb.load("train_train_level0.pkl.z")
train_train_level1 = jb.load("train_train_level1.pkl.z")
train_valid_level1 = jb.load("train_valid_level1.pkl.z")
test = pd.read_feather("test.f")


X_train0, y_train0 = train_train_level0.filter(regex=r'feature', axis=1), train_train_level0['target_kazutsugi']
X_train1, y_train1 = train_train_level1.filter(regex=r'feature', axis=1), train_train_level1['target_kazutsugi']
X_val1, y_val1 = train_valid_level1.filter(regex=r'feature', axis=1), train_valid_level1['target_kazutsugi']
X_test = test.filter(regex=r'feature', axis=1)

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class MLPRegressorTorch(nn.Module):
    def __init__(self, input_size, hidden1, hidden2):
        super(MLPRegressorTorch,self).__init__()
        self.hidden1 = nn.Linear(input_size, hidden1)
        self.hidden2 = nn.Linear(hidden1, hidden2)
        self.out = nn.Linear(hidden2, 1)
        
    def forward(self, x):
        x = F.relu(self.hidden1(x))
        x = F.relu(self.hidden2(x))
        return self.out(x)
    
    
#MLPRegressorTorch(X_train0.shape[1], 10, 10)(torch.from_numpy(X_train0.values).float())

In [22]:
from skopt import gp_minimize



X_train0_t = torch.from_numpy(X_train0.values).float()
y_train0_t = torch.from_numpy(y_train0.values).float().unsqueeze(dim=-1)

X_train1_t = torch.from_numpy(X_train1.values).float()
X_val1_t = torch.from_numpy(X_val1.values).float()
X_test_t = torch.from_numpy(X_test.values).float()


def tune_mlp(params):
    #scaling?
    torch.manual_seed(0)
    hidden1, hidden2, learning_rate = params
    mdl = MLPRegressorTorch(X_train0.shape[1], hidden1, hidden2)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(mdl.parameters(), lr=learning_rate)
    
    for epoch in range(100): 
        optimizer.zero_grad()

        p = mdl(X_train0_t)
        loss = criterion(p, y_train0_t)
        loss.backward()
        optimizer.step()
    
    p = mdl(X_train1_t).detach().numpy()
    model_name_train1 = "./preds_train1/mlp_{}_{}_{}.pkl.z".format(hidden1, hidden2, learning_rate) 
    jb.dump(p, model_name_train1)
    #print(p)
    
    metric = spearmanr(y_train1.values,p).correlation
    
    p = mdl(X_val1_t).detach().numpy()
    model_name_val1 = "./preds_val1/mlp_{}_{}_{}.pkl.z".format(hidden1, hidden2, learning_rate) 
    jb.dump(p, model_name_val1)
    
    p = mdl(X_test_t).detach().numpy()
    model_name_test = "./preds_test/mlp_{}_{}_{}.pkl.z".format(hidden1, hidden2, learning_rate) 
    jb.dump(p, model_name_test)
    
    print(params, metric)
    print()
    
    return -metric

In [23]:
tune_mlp([10,10,1e-4])

[10, 10, 0.0001] 0.0023646459357289417



-0.0023646459357289417

In [24]:
space = [(10, 200),
         (10, 200),
         (1e-5, 1e-3, 'log-uniform')]

res = gp_minimize(tune_mlp, space, random_state=0, verbose=1, n_calls=20)

Iteration No: 1 started. Evaluating function at random point.
[123, 170, 0.0005198657849887135] 0.01238510772187785

Iteration No: 1 ended. Evaluation done at random point.
Time taken: 27.9644
Function value obtained: -0.0124
Current minimum: -0.0124
Iteration No: 2 started. Evaluating function at random point.
[171, 128, 5.871694001325899e-05] 0.0007517081376493913

Iteration No: 2 ended. Evaluation done at random point.
Time taken: 28.6673
Function value obtained: -0.0008
Current minimum: -0.0124
Iteration No: 3 started. Evaluating function at random point.
[67, 21, 3.5100442309980525e-05] 0.0018467669241995371

Iteration No: 3 ended. Evaluation done at random point.
Time taken: 10.0310
Function value obtained: -0.0018
Current minimum: -0.0124
Iteration No: 4 started. Evaluating function at random point.
[101, 164, 9.119149691664945e-05] 0.005965180491993697

Iteration No: 4 ended. Evaluation done at random point.
Time taken: 26.5662
Function value obtained: -0.0060
Current minimum: 

In [ ]:
# fazer diário